<a href="https://colab.research.google.com/github/Fabiancaru/Advanced_Methods_Data_Analysis_II/blob/main/generacion_texto_RNN_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GENERACIÓN DE TEXTO CON REDES NEURONALES RECURRENTES

## 1 - El set de datos y el problema a resolver

Usaremos un texto en español, de libre acceso, y que contiene una serie de [cuentos de Edgar Allan Poe](https://www.gutenberg.org/cache/epub/46196/pg46196.txt).

La idea es crear un modelo de lenguaje a nivel de caracter: para cada predicción tomará 10 caracteres de entrada y generará 1 de salida:

![](https://drive.google.com/uc?export=view&id=1UqXixOnlp4AS7gL1EZiZW0J1qJ9ctg3B)

In [1]:
# Las librerías a utilizar
import numpy as np
from tensorflow.keras.layers import Dense, SimpleRNN
from keras.models import Sequential
from google.colab import drive
import tensorflow as tf

In [10]:
# Leer archivo
drive.mount('/gdrive')
ruta = '/gdrive/MyDrive/Colab Notebooks/cuentos_EAPoe.txt'

texto = open(ruta,'r',encoding='utf8').read()
chars = list(set(texto))
tam_vocab = len(chars)

print(texto)
print(chars)
print(tam_vocab)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
EL BARRIL DE AMONTILLADO


HABÍA soportado lo mejor posible los mil pequeños agravios de Fortunato;
pero cuando se atrevió a llegar hasta el ultraje, juré que había de
vengarme. Vosotros, que tan bien conocéis mi temperamento, no supondréis
que pronuncié la más ligera amenaza. _Algún día_ me vengaría; esto era
definitivo; pero la misma decisión que abrigaba, excluía toda idea de
correr el menor riesgo. No solamente era necesario castigar, sino
castigar con impunidad. No se repara un agravio cuando la reparación se
vuelve en contra del justiciero; ni tampoco se repara cuando no se hace
sentir al ofensor de qué parte proviene el castigo.

Es necesario tener presente que jamás había dado a Fortunato, ni por
medio de palabras ni de acciones, ocasión de sospechar de mi buena
voluntad. Continué sonriéndole siempre, como era mi deseo, y él no se
apercibió de que _ahora_ sonreía y

## 2 - Creación del set de entrenamiento

In [11]:
# Diccionarios para convertir de texto a índice numérico y viceversa
char2ix = {c: i for i, c in enumerate(chars)}
ix2char = {i: c for i, c in enumerate(chars)}
print(char2ix)
print(ix2char)

{'T': 0, 'É': 1, 'F': 2, 'Q': 3, 'ö': 4, 'Ú': 5, '5': 6, 'M': 7, 'P': 8, 'I': 9, '{': 10, 'c': 11, 'á': 12, 'ñ': 13, 'G': 14, 'B': 15, '1': 16, 'í': 17, '"': 18, 'n': 19, 'x': 20, ' ': 21, 'K': 22, '?': 23, 'ó': 24, '¿': 25, ']': 26, 'Ó': 27, 'R': 28, 'y': 29, '6': 30, 'v': 31, 'q': 32, '¡': 33, 'p': 34, 'i': 35, 'W': 36, '(': 37, '[': 38, '_': 39, '¶': 40, 'L': 41, 'U': 42, 'N': 43, 'f': 44, 'E': 45, 'h': 46, 'l': 47, 'e': 48, 'z': 49, 'ú': 50, '4': 51, ',': 52, 'k': 53, 'V': 54, '-': 55, 'æ': 56, 'a': 57, 'X': 58, '\n': 59, '}': 60, '0': 61, '2': 62, 'A': 63, ':': 64, '7': 65, '.': 66, 'Ö': 67, ')': 68, '8': 69, 't': 70, 'm': 71, 'D': 72, 'Á': 73, '*': 74, 's': 75, 'â': 76, 'H': 77, 'C': 78, 'Y': 79, 'J': 80, 'Í': 81, 'g': 82, '!': 83, ';': 84, 'é': 85, 'ü': 86, 'u': 87, 'w': 88, '3': 89, 'ô': 90, 'o': 91, 'd': 92, "'": 93, 'j': 94, '9': 95, 'b': 96, 'r': 97, 'O': 98, 'S': 99}
{0: 'T', 1: 'É', 2: 'F', 3: 'Q', 4: 'ö', 5: 'Ú', 6: '5', 7: 'M', 8: 'P', 9: 'I', 10: '{', 11: 'c', 12: 'á', 

In [12]:
# Crear bloques de 10 caracteres, que serán los datos de entrada a la red
LONG_SEC = 10
texto_in, texto_out = [], []

# Por cada 10 caracteres de entrada, el modelo predice 1 caracter de salida
for i in range(0,len(texto)-LONG_SEC):
  texto_in.append(texto[i:i+LONG_SEC]) # Ejemplo: caracteres del 0 al 9
  texto_out.append(texto[i + LONG_SEC])# Ejemplo: caracter 10

print(texto_in[0])
print(texto_out[0])
print(texto_in[1])
print(texto_out[1])

EL BARRIL 
D
L BARRIL D
E


In [13]:
# Set de entrenamiento: simplemente se deben convertir a one-hot los sets
# de entrada y salida

X = np.zeros((len(texto_in), LONG_SEC, tam_vocab)) # #ejemplos x 10 x tam_vocab
Y = np.zeros((len(texto_in), tam_vocab))   # #ejemplos x tam_vocab

for i, entrada in enumerate(texto_in):
  for j, car in enumerate(entrada):
    X[i,j, char2ix[car]] = 1
    Y[i, char2ix[texto_out[i]]] = 1

print(X.shape)
print(Y.shape)
print(X[0,0,:])
print(Y[0,:])

(331185, 10, 100)
(331185, 100)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


## 3 - Crear modelo

In [14]:
SEED = 15
tf.random.set_seed(SEED)
np.random.seed(SEED)
N_NEURONAS = 256

modelo = Sequential()
modelo.add( SimpleRNN(N_NEURONAS, input_shape=(LONG_SEC, tam_vocab)) )
modelo.add(Dense(tam_vocab, activation='softmax'))

modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 256)               91392     
                                                                 
 dense (Dense)               (None, 100)               25700     
                                                                 
Total params: 117,092
Trainable params: 117,092
Non-trainable params: 0
_________________________________________________________________


## 4 - Función para generación de texto

In [15]:
def generar_texto(modelo, L_OUT_SEC):
  # Seleccionar un dato de entrenamiento inicial aleatoriamente
  ix_test = np.random.randint(len(texto_in))
  char_test = texto_in[ix_test]

  # Generar texto de tamaño L_OUT_SEC
  print(char_test, end="")
  for i in range(L_OUT_SEC):
    # Codificación one-hot del texto de entrada
    X_test = np.zeros((1, LONG_SEC, tam_vocab)) # 1x10x100
    for j, car in enumerate(char_test):
      X_test[0,j, char2ix[car]] = 1
    
    # Introducir al modelo y generar predicción
    pred = modelo.predict(X_test, verbose=0)[0]
    y_pred = ix2char[np.argmax(pred)]

    # Imprimir el caracter predicho
    print(y_pred, end="")

    # Agregar como último elemento de "char_test" y continuar prediciendo
    char_test = char_test[1:] + y_pred
  print()

## 5 - Entrenamiento y generación de texto

In [16]:
modelo.compile(loss="categorical_crossentropy", optimizer="rmsprop")

NEPOCHS = 20
BATCH_SIZE = 128
L_OUT_SEC = 50      # Número de caracteres a generar

print('-'*50)
print('Antes del entrenamiento:')
generar_texto(modelo, L_OUT_SEC)

for it in range(NEPOCHS):
  modelo.fit(X,Y, batch_size=BATCH_SIZE, epochs=1, verbose=0)
  print('-'*50)
  print(f'Iteración: {it+1}')
  generar_texto(modelo, L_OUT_SEC)

--------------------------------------------------
Antes del entrenamiento:
aba y acarn?üxO82Ózvt}:nA;¶Y;U8pV3üuL¶)T kÖ ,Ö4_ürhüô_ âIRRz
--------------------------------------------------
Iteración: 1
 ruin amista de la de la de la de la de la de la de la de la
--------------------------------------------------
Iteración: 2
 doble Dupara de la con el por en en esta mente a su vin en 
--------------------------------------------------
Iteración: 3
eza, mucho de la veza de la cama a su ventan en la mis a de 
--------------------------------------------------
Iteración: 4
 pausada y esta cama se presentamente a mi abre de la viste 
--------------------------------------------------
Iteración: 5
s. Legrando de la viste al sunto de desparecía en la cama a 
--------------------------------------------------
Iteración: 6
encontrábando de la vistan esta manera de la casa con el per
--------------------------------------------------
Iteración: 7
respecto?--de su vez no de aquella de despresenta